In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import json

In [6]:

def main(code):
    res = requests.get(BASE_URL + str(code) + '&page=' + str(START_PAGE))
    print(res)
    page_soup = BeautifulSoup(res.text, 'lxml')
    
    total_page_num = page_soup.select_one('td.pgRR > a')
    print(total_page_num)
    total_page_num = int(total_page_num.get('href').split('=')[-1])
    print(total_page_num)

    ipt_html = page_soup.select_one('div.subcnt_sise_item_top')
    global fields
    fields = [item.get('value') for item in ipt_html.select('input')]

    result = [crawl(code,str(page)) for page in range(1, total_page_num + 1)]

    df = pd.concat(result, axis = 0, ignore_index = True)
    return df

def crawl(code, page):
    global fields
    data = {'menu': 'market_sum',
            'fieldIds': fields,
            'returnUrl': BASE_URL + str(code) + "&page=" + str(page)}

    res = requests.post('https://finance.naver.com/sise/field_submit.nhn', data = data)
    page_soup = BeautifulSoup(res.text, 'lxml')

    table_html = page_soup.select_one('div.box_type_l')

    header_data = [item.get_text().strip() for item in table_html.select('thead th')][1:-1]

    inner_data = [item.get_text().strip() for item in table_html.find_all(lambda x:
                                                                            (x.name == 'a' and
                                                                             'tltle' in x.get('class', [])) or
                                                                            (x.name == 'td' and
                                                                             'number' in x.get('class', []))
                                                                            )]
    no_data = [item.get_text().strip() for item in table_html.select('td.no')]
    number_data = np.array(inner_data)

    number_data.resize(len(no_data), len(header_data))

    df = pd.DataFrame(data = number_data, columns = header_data)
    return df


BASE_URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='
KOSPI_CODE = 0
KOSDAK_CODE = 1
START_PAGE = 1
field = []
KOSPI = main(KOSPI_CODE)
KOSPI
# KOSDAK = main(KOSDAK_CODE)
# KOSPI = KOSPI.replace('N/A',np.nan).dropna(axis = 0)
# KOSPI['PER'] = KOSPI['PER'].str.replace(',','').astype('float64')
# KOSPI['ROE'] = KOSPI['ROE'].str.replace(',','').astype('float64')
# KOSPI['ROA'] = KOSPI['ROA'].str.replace(',','').astype('float64')
# KOSPI['시가총액'] = KOSPI['시가총액'].str.replace(',','').astype('int64')
# KOSPI['매출액'] = KOSPI['매출액'].str.replace(',','').astype('int64')
# KOSPI['영업이익'] = KOSPI['영업이익'].str.replace(',','').astype('int64')
# KOSPI['PSR'] = KOSPI['시가총액'] / KOSPI['매출액']
# KOSPI['POR'] = KOSPI['시가총액'] / KOSPI['영업이익']
# KOSPI = KOSPI[['종목명','PER','ROE','ROA','PBR','PSR','POR']]
# c = ['PER','POR']
# KOSPI[c] = KOSPI[c].mask(KOSPI[c] < 0)
# KOSPI = KOSPI.dropna(axis = 0)
# KOSPI['PBR + PSR'] = KOSPI[['PBR','PSR']].rank(method = 'min').sum(axis = 1)
# KOSPI['PBR + PSR + POR'] = KOSPI[['PBR','PSR','POR']].rank(method = 'min').sum(axis = 1)
# KOSPI['total_rank'] = (KOSPI[['PER','PBR','PSR']].rank(method = 'min').sum(axis = 1) + KOSPI[['ROE','ROA']].rank(method = 'min',ascending = False).sum(axis = 1))
# PBR_PSR = KOSPI[['종목명','PBR','PSR','PBR + PSR']].sort_values(by = 'PBR + PSR')
# PBR_PSR = PBR_PSR.drop(['PBR + PSR'],axis = 1)
# PBR_PSR.index = list(range(1,len(PBR_PSR)+1))
# df = PBR_PSR.copy()
# json_records = df.reset_index().to_json(orient ='records') 
# data = [] 
# data = json.loads(json_records) 
# context = {'d': data} 

# return render(request, 'selnm/pbr_psr.html', context) 


<Response [200]>
<a href="/sise/sise_market_sum.nhn?sosok=0&amp;page=32">맨뒤
				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
32


,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,주당순이익,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율
0,삼성전자,"59,400","1,000",+1.71%,100,"14,609,079","871,308","30,597,399","59,900","60,000",...,"3,196","1,416",-5.49,-52.84,55.90,18.59,8.69,6.28,1.54,"28,856.0"
1,SK하이닉스,"76,900",400,-0.52%,"5,000","1,319,820","102,236","3,918,435","78,000","78,200",...,"3,140","1,000",-33.27,-86.99,47.22,24.49,4.25,3.14,1.07,"1,287.0"
2,삼성바이오로직스,"765,000","6,000",+0.79%,"2,500","39,749","30,310","129,170","766,000","767,000",...,"5,191",N/A,30.94,64.77,10.24,147.37,4.77,3.41,11.39,"2,539.1"
3,NAVER,"305,500","1,000",+0.33%,100,"348,191","107,220","925,825","309,000","311,000",...,"4,290",376,18.02,-24.66,55.13,71.21,10.56,3.58,6.42,"44,216.1"
4,LG화학,"702,000","3,000",+0.43%,"5,000","236,266","166,945","575,376","716,000","716,000",...,"5,801","2,000",1.57,-60.12,35.95,121.01,1.84,1.20,3.10,"4,362.1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,KBSTAR 200건설,"8,290",65,+0.79%,0,2,0,48,"8,290","8,290",...,N/A,N/A,N/A,N/A,0.00,N/A,N/A,N/A,N/A,N/A
1561,ARIRANG 200동일가중,"8,595",20,+0.23%,0,2,0,60,"8,605","8,605",...,N/A,N/A,N/A,N/A,0.00,N/A,N/A,N/A,N/A,N/A
1562,KBSTAR 200금융,"6,630",30,+0.45%,0,120,0,277,"6,695","6,695",...,N/A,N/A,N/A,N/A,0.00,N/A,N/A,N/A,N/A,N/A
1563,KBSTAR 200경기소비재,"8,485",25,+0.30%,0,1,0,20,"8,485","8,485",...,N/A,N/A,N/A,N/A,0.00,N/A,N/A,N/A,N/A,N/A
